<a href="https://colab.research.google.com/github/RiteshopShrivastava/Neural_Network/blob/main/Assignment_NN_ForestFires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV, KFold

In [6]:
data = pd.read_csv('forestfires.csv')

In [7]:
data.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small


In [8]:
data1= data.drop(['month','day'], axis=1)

In [9]:
data1

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,small
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,small
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,large
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,large
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,large
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,small


In [15]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data1['size_category']= label_encoder.fit_transform(data1['size_category'])

In [16]:
data1

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [18]:
X= data1.iloc[:,0:28]
Y= data1.iloc[:,28:]

In [11]:
X

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [19]:
Y

,size_category
0,1
1,1
2,1
3,1
4,1
...,...
512,0
513,0
514,0
515,1


In [21]:
scaler= StandardScaler()
scaler.fit(X)
X_standarized = scaler.transform(X)

In [22]:
X_standarized

array([[-8.05959472e-01, -1.32332557e+00, -1.83047676e+00, ...,
        -4.40225453e-02, -1.72859706e-01, -7.06081245e-01],
       [-8.10203395e-03, -1.17954077e+00,  4.88890915e-01, ...,
        -4.40225453e-02,  5.78503817e+00, -7.06081245e-01],
       [-8.10203395e-03, -1.04982188e+00,  5.60715454e-01, ...,
        -4.40225453e-02,  5.78503817e+00, -7.06081245e-01],
       ...,
       [-1.64008316e+00, -8.46647711e-01,  4.74768113e-01, ...,
        -4.40225453e-02, -1.72859706e-01, -7.06081245e-01],
       [ 6.80956663e-01,  5.49002541e-01,  2.69382214e-01, ...,
        -4.40225453e-02, -1.72859706e-01, -7.06081245e-01],
       [-2.02087875e+00, -1.68591332e+00, -1.78044169e+00, ...,
         2.27156334e+01, -1.72859706e-01, -7.06081245e-01]])

In [26]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout
from sklearn.metrics import accuracy_score

**Tuning of Hyperparameter : Batch size and Epoch**

In [37]:
def create_model():
  model= Sequential()
  model.add(Dense(8,input_dim=28, kernel_initializer='uniform', activation='relu'))
  model.add(Dropout(0.01))
  model.add(Dense(4, kernel_initializer='uniform', activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
  adam= Adam(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

model= KerasClassifier(build_fn=create_model, verbose=0)

batch_size= [10,30,50]
epochs = [10,20,30]
param_grid = dict( batch_size= batch_size, epochs= epochs)
gsv = GridSearchCV( estimator=model, param_grid=param_grid, cv=KFold(), verbose=5)
gsv_result = gsv.fit(X_standarized, Y)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ batch_size=10, epochs=10, score=0.990, total=   1.3s
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


[CV] ............ batch_size=10, epochs=10, score=0.750, total=   1.1s
[CV] batch_size=10, epochs=10 ........................................


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.7s remaining:    0.0s


[CV] ............ batch_size=10, epochs=10, score=0.689, total=   1.1s
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.8s remaining:    0.0s


[CV] ............ batch_size=10, epochs=10, score=0.699, total=   1.3s
[CV] batch_size=10, epochs=20 ........................................
[CV] ............ batch_size=10, epochs=20, score=0.990, total=   1.9s
[CV] batch_size=10, epochs=20 ........................................
[CV] ............ batch_size=10, epochs=20, score=0.750, total=   1.6s
[CV] batch_size=10, epochs=20 ........................................
[CV] ............ batch_size=10, epochs=20, score=0.524, total=   1.9s
[CV] batch_size=10, epochs=20 ........................................
[CV] ............ batch_size=10, epochs=20, score=0.699, total=   1.9s
[CV] batch_size=10, epochs=20 ........................................
[CV] ............ batch_size=10, epochs=20, score=0.777, total=   1.9s
[CV] batch_size=10, epochs=30 ........................................
[CV] ............ batch_size=10, epochs=30, score=1.000, total=   3.2s
[CV] batch_size=10, epochs=30 ........................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   59.8s finished


In [38]:
print(gsv_result.best_score_, gsv_result.best_params_)

0.827613890171051 {'batch_size': 10, 'epochs': 30}


**Tuning of Hyperparameter : Learning rate & Dropout rate**

In [39]:
def create_model(learning_rate, dropout_rate):
  model= Sequential()
  model.add(Dense(8,input_dim=28, kernel_initializer='uniform', activation='relu'))
  model.add(Dropout(0.01))
  model.add(Dense(4, kernel_initializer='uniform', activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
  adam= Adam(learning_rate=learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

model= KerasClassifier(build_fn=create_model, batch_size=10, epochs=30, verbose=0)

learning_rate= [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]
param_grid = dict( learning_rate=learning_rate, dropout_rate= dropout_rate)
gsv = GridSearchCV( estimator=model, param_grid=param_grid, cv=KFold(), verbose=5)
gsv_result = gsv.fit(X_standarized, Y)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] dropout_rate=0.0, learning_rate=0.001 ...........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  dropout_rate=0.0, learning_rate=0.001, score=1.000, total=   3.2s
[CV] dropout_rate=0.0, learning_rate=0.001 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s


[CV]  dropout_rate=0.0, learning_rate=0.001, score=0.769, total=   3.2s
[CV] dropout_rate=0.0, learning_rate=0.001 ...........................


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    8.5s remaining:    0.0s


[CV]  dropout_rate=0.0, learning_rate=0.001, score=0.786, total=   2.1s
[CV] dropout_rate=0.0, learning_rate=0.001 ...........................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   10.6s remaining:    0.0s


[CV]  dropout_rate=0.0, learning_rate=0.001, score=0.893, total=   2.1s
[CV] dropout_rate=0.0, learning_rate=0.01 ............................
[CV]  dropout_rate=0.0, learning_rate=0.01, score=1.000, total=   2.1s
[CV] dropout_rate=0.0, learning_rate=0.01 ............................
[CV]  dropout_rate=0.0, learning_rate=0.01, score=0.913, total=   3.2s
[CV] dropout_rate=0.0, learning_rate=0.01 ............................
[CV]  dropout_rate=0.0, learning_rate=0.01, score=0.883, total=   2.4s
[CV] dropout_rate=0.0, learning_rate=0.01 ............................
[CV]  dropout_rate=0.0, learning_rate=0.01, score=0.913, total=   2.1s
[CV] dropout_rate=0.0, learning_rate=0.01 ............................
[CV]  dropout_rate=0.0, learning_rate=0.01, score=0.913, total=   2.1s
[CV] dropout_rate=0.0, learning_rate=0.1 .............................
[CV] . dropout_rate=0.0, learning_rate=0.1, score=1.000, total=   2.1s
[CV] dropout_rate=0.0, learning_rate=0.1 .............................
[CV] 

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.9min finished


In [40]:
print(gsv_result.best_score_, gsv_result.best_params_)

0.9244585514068604 {'dropout_rate': 0.2, 'learning_rate': 0.01}


Tuning of Hyperparameter :Activation Function & Kernel Initializer

In [41]:
def create_model(activation_function, init):
  model= Sequential()
  model.add(Dense(8,input_dim=28, kernel_initializer='uniform', activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(4, kernel_initializer='uniform', activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
  adam= Adam(learning_rate=0.01)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

model= KerasClassifier(build_fn=create_model, batch_size=10, epochs=30, verbose=0)

activation_function= ['relu', 'tanh','softmax','linear']
init = ['zero', 'uniform','normal']
param_grid = dict( activation_function=activation_function, init= init)
gsv = GridSearchCV( estimator=model, param_grid=param_grid, cv=KFold(), verbose=5)
gsv_result = gsv.fit(X_standarized, Y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] activation_function=relu, init=zero .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . activation_function=relu, init=zero, score=1.000, total=   3.2s
[CV] activation_function=relu, init=zero .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s


[CV] . activation_function=relu, init=zero, score=0.865, total=   3.2s
[CV] activation_function=relu, init=zero .............................


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    8.5s remaining:    0.0s


[CV] . activation_function=relu, init=zero, score=0.913, total=   2.2s
[CV] activation_function=relu, init=zero .............................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   10.6s remaining:    0.0s


[CV] . activation_function=relu, init=zero, score=0.913, total=   2.1s
[CV] activation_function=relu, init=uniform ..........................
[CV]  activation_function=relu, init=uniform, score=1.000, total=   3.2s
[CV] activation_function=relu, init=uniform ..........................
[CV]  activation_function=relu, init=uniform, score=0.856, total=   3.2s
[CV] activation_function=relu, init=uniform ..........................
[CV]  activation_function=relu, init=uniform, score=0.903, total=   2.1s
[CV] activation_function=relu, init=uniform ..........................
[CV]  activation_function=relu, init=uniform, score=0.883, total=   3.2s
[CV] activation_function=relu, init=uniform ..........................
[CV]  activation_function=relu, init=uniform, score=0.913, total=   2.4s
[CV] activation_function=relu, init=normal ...........................
[CV]  activation_function=relu, init=normal, score=1.000, total=   3.2s
[CV] activation_function=relu, init=normal .......................

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  2.6min finished


In [43]:
print(gsv_result.best_score_, gsv_result.best_params_)

0.9225354790687561 {'activation_function': 'softmax', 'init': 'normal'}


In [44]:
def create_model(neuron1, neuron2):
  model= Sequential()
  model.add(Dense(neuron1,input_dim=28, kernel_initializer='normal', activation='softmax'))
  model.add(Dropout(0.2))
  model.add(Dense(neuron2, kernel_initializer='normal', activation='softmax'))
  model.add(Dropout(0.2))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  adam= Adam(learning_rate=0.01)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

model= KerasClassifier(build_fn=create_model, batch_size=10, epochs=30, verbose=0)

neuron1= [24,16,8]
neuron2= [12,8,4]
param_grid = dict( neuron1=neuron1, neuron2= neuron2)
gsv = GridSearchCV( estimator=model, param_grid=param_grid, cv=KFold(), verbose=5)
gsv_result = gsv.fit(X_standarized, Y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] neuron1=24, neuron2=12 ..........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .............. neuron1=24, neuron2=12, score=0.971, total=   2.3s
[CV] neuron1=24, neuron2=12 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV] .............. neuron1=24, neuron2=12, score=0.856, total=   3.2s
[CV] neuron1=24, neuron2=12 ..........................................


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.8s remaining:    0.0s


[CV] .............. neuron1=24, neuron2=12, score=0.854, total=   3.2s
[CV] neuron1=24, neuron2=12 ..........................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   11.0s remaining:    0.0s


[CV] .............. neuron1=24, neuron2=12, score=0.835, total=   3.2s
[CV] neuron1=24, neuron2=8 ...........................................
[CV] ............... neuron1=24, neuron2=8, score=0.990, total=   2.7s
[CV] neuron1=24, neuron2=8 ...........................................
[CV] ............... neuron1=24, neuron2=8, score=0.837, total=   3.2s
[CV] neuron1=24, neuron2=8 ...........................................
[CV] ............... neuron1=24, neuron2=8, score=0.738, total=   2.3s
[CV] neuron1=24, neuron2=8 ...........................................
[CV] ............... neuron1=24, neuron2=8, score=0.845, total=   3.2s
[CV] neuron1=24, neuron2=8 ...........................................
[CV] ............... neuron1=24, neuron2=8, score=0.854, total=   3.2s
[CV] neuron1=24, neuron2=4 ...........................................
[CV] ............... neuron1=24, neuron2=4, score=0.990, total=   3.2s
[CV] neuron1=24, neuron2=4 ...........................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  2.1min finished


In [45]:
print(gsv_result.best_score_, gsv_result.best_params_)

0.8857356309890747 {'neuron1': 16, 'neuron2': 12}


**Training model with optimum values of Hyperparameters**

In [48]:
def create_model():
  model= Sequential()
  model.add(Dense(16,input_dim=28, kernel_initializer='normal', activation='softmax'))
  model.add(Dropout(0.2))
  model.add(Dense(12, kernel_initializer='normal', activation='softmax'))
  model.add(Dropout(0.2))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  adam= Adam(learning_rate=0.01)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

model= KerasClassifier(build_fn=create_model, batch_size=10, epochs=30, verbose=0)
model.fit(X_standarized, Y)
y_predict = model.predict(X_standarized)
print('Accuracy :',accuracy_score(Y, y_predict))

Accuracy : 0.9806576402321083
